# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe6b8f5e7f0>
├── 'a' --> tensor([[-0.4175,  0.2473,  0.6786],
│                   [-2.8303, -0.1946, -0.0074]])
└── 'x' --> <FastTreeValue 0x7fe6b8f5ea90>
    └── 'c' --> tensor([[-1.1471,  0.7792,  0.2892, -0.6883],
                        [ 0.1448, -1.3231,  0.2050, -0.3475],
                        [-0.5098, -2.1972, -0.1330,  1.1255]])

In [4]:
t.a

tensor([[-0.4175,  0.2473,  0.6786],
        [-2.8303, -0.1946, -0.0074]])

In [5]:
%timeit t.a

59.9 ns ± 0.0459 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe6b8f5e7f0>
├── 'a' --> tensor([[-0.5542,  1.0444, -1.3078],
│                   [-0.5230, -1.2510,  1.9229]])
└── 'x' --> <FastTreeValue 0x7fe6b8f5ea90>
    └── 'c' --> tensor([[-1.1471,  0.7792,  0.2892, -0.6883],
                        [ 0.1448, -1.3231,  0.2050, -0.3475],
                        [-0.5098, -2.1972, -0.1330,  1.1255]])

In [7]:
%timeit t.a = new_value

61.9 ns ± 0.0533 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4175,  0.2473,  0.6786],
               [-2.8303, -0.1946, -0.0074]]),
    x: Batch(
           c: tensor([[-1.1471,  0.7792,  0.2892, -0.6883],
                      [ 0.1448, -1.3231,  0.2050, -0.3475],
                      [-0.5098, -2.1972, -0.1330,  1.1255]]),
       ),
)

In [10]:
b.a

tensor([[-0.4175,  0.2473,  0.6786],
        [-2.8303, -0.1946, -0.0074]])

In [11]:
%timeit b.a

59.7 ns ± 0.0213 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.8666,  1.0278,  0.4811],
               [-0.6435,  0.5205, -0.0104]]),
    x: Batch(
           c: tensor([[-1.1471,  0.7792,  0.2892, -0.6883],
                      [ 0.1448, -1.3231,  0.2050, -0.3475],
                      [-0.5098, -2.1972, -0.1330,  1.1255]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 1.82 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

845 ns ± 0.301 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 36.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 4.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe60ebf9310>
├── 'a' --> tensor([[[-0.4175,  0.2473,  0.6786],
│                    [-2.8303, -0.1946, -0.0074]],
│           
│                   [[-0.4175,  0.2473,  0.6786],
│                    [-2.8303, -0.1946, -0.0074]],
│           
│                   [[-0.4175,  0.2473,  0.6786],
│                    [-2.8303, -0.1946, -0.0074]],
│           
│                   [[-0.4175,  0.2473,  0.6786],
│                    [-2.8303, -0.1946, -0.0074]],
│           
│                   [[-0.4175,  0.2473,  0.6786],
│                    [-2.8303, -0.1946, -0.0074]],
│           
│                   [[-0.4175,  0.2473,  0.6786],
│                    [-2.8303, -0.1946, -0.0074]],
│           
│                   [[-0.4175,  0.2473,  0.6786],
│                    [-2.8303, -0.1946, -0.0074]],
│           
│                   [[-0.4175,  0.2473,  0.6786],
│                    [-2.8303, -0.1946, -0.0074]]])
└── 'x' --> <FastTreeValue 0x7fe60ec11f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 40.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe60ebf95e0>
├── 'a' --> tensor([[-0.4175,  0.2473,  0.6786],
│                   [-2.8303, -0.1946, -0.0074],
│                   [-0.4175,  0.2473,  0.6786],
│                   [-2.8303, -0.1946, -0.0074],
│                   [-0.4175,  0.2473,  0.6786],
│                   [-2.8303, -0.1946, -0.0074],
│                   [-0.4175,  0.2473,  0.6786],
│                   [-2.8303, -0.1946, -0.0074],
│                   [-0.4175,  0.2473,  0.6786],
│                   [-2.8303, -0.1946, -0.0074],
│                   [-0.4175,  0.2473,  0.6786],
│                   [-2.8303, -0.1946, -0.0074],
│                   [-0.4175,  0.2473,  0.6786],
│                   [-2.8303, -0.1946, -0.0074],
│                   [-0.4175,  0.2473,  0.6786],
│                   [-2.8303, -0.1946, -0.0074]])
└── 'x' --> <FastTreeValue 0x7fe60ebf9760>
    └── 'c' --> tensor([[-1.1471,  0.7792,  0.2892, -0.6883],
                        [ 0.1448, -1.3231,  0.2050, -0.3475],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 59.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.2 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.1471,  0.7792,  0.2892, -0.6883],
                       [ 0.1448, -1.3231,  0.2050, -0.3475],
                       [-0.5098, -2.1972, -0.1330,  1.1255]],
              
                      [[-1.1471,  0.7792,  0.2892, -0.6883],
                       [ 0.1448, -1.3231,  0.2050, -0.3475],
                       [-0.5098, -2.1972, -0.1330,  1.1255]],
              
                      [[-1.1471,  0.7792,  0.2892, -0.6883],
                       [ 0.1448, -1.3231,  0.2050, -0.3475],
                       [-0.5098, -2.1972, -0.1330,  1.1255]],
              
                      [[-1.1471,  0.7792,  0.2892, -0.6883],
                       [ 0.1448, -1.3231,  0.2050, -0.3475],
                       [-0.5098, -2.1972, -0.1330,  1.1255]],
              
                      [[-1.1471,  0.7792,  0.2892, -0.6883],
                       [ 0.1448, -1.3231,  0.2050, -0.3475],
                       [-0.5098, -2.1972, -0.1330,  1.1255]],

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 256 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.1471,  0.7792,  0.2892, -0.6883],
                      [ 0.1448, -1.3231,  0.2050, -0.3475],
                      [-0.5098, -2.1972, -0.1330,  1.1255],
                      [-1.1471,  0.7792,  0.2892, -0.6883],
                      [ 0.1448, -1.3231,  0.2050, -0.3475],
                      [-0.5098, -2.1972, -0.1330,  1.1255],
                      [-1.1471,  0.7792,  0.2892, -0.6883],
                      [ 0.1448, -1.3231,  0.2050, -0.3475],
                      [-0.5098, -2.1972, -0.1330,  1.1255],
                      [-1.1471,  0.7792,  0.2892, -0.6883],
                      [ 0.1448, -1.3231,  0.2050, -0.3475],
                      [-0.5098, -2.1972, -0.1330,  1.1255],
                      [-1.1471,  0.7792,  0.2892, -0.6883],
                      [ 0.1448, -1.3231,  0.2050, -0.3475],
                      [-0.5098, -2.1972, -0.1330,  1.1255],
                      [-1.1471,  0.7792,  0.2892, -0.6883],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 180 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 871 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
